In [38]:
#analysis and scraping
import pandas as pd
import requests
from bs4 import BeautifulSoup
from string import ascii_lowercase
import time
import ast
#import json 




#

## plotting libraries
import matplotlib.pyplot as plt
import pandas as pd

# Get data

In [9]:
#Input
root_kw = "why are bicycles "
keywords = [root_kw + c for c in ascii_lowercase]
country = "uk" 

In [13]:
#get all suggestions from Google
data = []
for kw in keywords:
    r = requests.get('http://suggestqueries.google.com/complete/search?output=toolbar&hl={}&q={}'.format(country,kw))
    soup = BeautifulSoup(r.content, 'html.parser')
    sugg = [sugg['data'] for sugg in soup.find_all('suggestion')]
    data.append(sugg[0:2])

In [14]:
data

[['why are bicycles allowed on the road', 'why are bicycles always exhausted'],
 ['why are bicycles better than cars', 'why are bicycles bad'],
 ['why are bicycles called eco-friendly vehicles', 'why are bikes cool'],
 ['why are bicycles dangerous', 'why are bikes dangerous'],
 ['why are bicycles expensive', 'why are electric bicycles used'],
 ['why are bikes fun', 'what are folding bicycle for'],
 ['why are bicycles good', 'why are bicycle good'],
 ['why are bicycle handlebars', 'why are bike helmet'],
 ['why are bicycles important', 'why are bicycles important in the giver'],
 ['why bicycle jersey'],
 ['what are bicycle kicks', 'what are bicycle kick'],
 ['why are bicycle lights', 'why are bike lanes important'],
 ['why are bicycles more expensive than motorcycles',
  'what are bicycle made of'],
 ['why are bicycles not suitable for all types of orders',
  'why bikes are not allowed on expressways'],
 ['why are specialized bikes on sale', 'why bike oil'],
 ['why are bicycles prohibit

In [15]:
#Input
root_kw = "why are cars "
keywords = [root_kw + c for c in ascii_lowercase]
country = "uk" 

In [17]:
#get all suggestions from Google
data = []
for kw in keywords:
    r = requests.get('http://suggestqueries.google.com/complete/search?output=toolbar&hl={}&q={}'.format(country,kw))
    soup = BeautifulSoup(r.content, 'html.parser')
    sugg = [sugg['data'] for sugg in soup.find_all('suggestion')]
    data.append(sugg[0:2])
    time.sleep(0.2)

In [18]:
data

[['why are cars abandoned in dubai',
  'why are cars aeroplanes and rockets streamlined'],
 ['why are cars bad for the environment', 'why are cars better than trucks'],
 ['why are cars called cars', 'why are cars called whips'],
 ['why are cars designed to crumple', 'why are cars dangerous'],
 ['why are cars expensive in the netherlands', 'why are cars expensive'],
 ['why are cars female', 'why are cars front wheel drive'],
 ['why are cars getting bigger', 'why are cars going electric'],
 ['why are cars harmful to the environment', 'why are cars helpful'],
 ['why are cars important', 'why are cars in cuba so old'],
 ['why are jdm cars popular', 'why are japanese cars cheap'],
 ['why are cars keyless', 'what are kit cars'],
 ['why are cars left hand drive', 'why are luxury cars cheap'],
 ['why are cars more expensive now',
  'why are cars made to go over the speed limit'],
 ['why are cars named after females', 'why are cars not made of aluminum'],
 ['why are cars on facebook marketplace

In [132]:
#Input
cyc_root = "why do cyclists "
cyc_keywords = [cyc_root + c for c in ascii_lowercase]

driver_root = "why do drivers "
driver_keywords = [driver_root + c for c in ascii_lowercase]

all_keywords = {"cyclists": cyc_keywords, "drivers" :driver_keywords}
country = "uk" 

In [133]:
#get all suggestions from Google
data = { "cyclists" : [], "drivers":[]}

for roaduser in data.keys():
    keywords = all_keywords[roaduser]
    for kw in keywords:
        r = requests.get('http://suggestqueries.google.com/complete/search?output=toolbar&hl={}&q={}'.format(country,kw))
        soup = BeautifulSoup(r.content, 'html.parser')
        sugg = [sugg['data'] for sugg in soup.find_all('suggestion')]
        data[roaduser].append(sugg[0:12])
        time.sleep(0.2)

In [137]:
df = pd.DataFrame.from_dict(data)
df["letter"] = [c.upper() for c in ascii_lowercase]
df = df.rename(columns={"cyclists":"cyclists_raw", "drivers":"drivers_raw"})

In [138]:
#get one entry
df["cyclists"]=df["cyclists_raw"].str[0]
df["cyclists"].iloc[5] = ""
df["cyclists"].iloc[10] = df["cyclists_raw"].iloc[10][6]
df["cyclists"].iloc[12] = ""
df["cyclists"].iloc[14] = "why do cyclists only have one headlight on"
df["cyclists"].iloc[16] = df["cyclists_raw"].iloc[16][3]
df["cyclists"].iloc[23] = ""
df["cyclists"].iloc[24] =  df["cyclists_raw"].iloc[24][5]

df["drivers"]=df["drivers_raw"].str[0]
#df["drivers"].iloc[0] = ""
#df["drivers"].iloc[9] = ""

In [139]:
df["drivers"]

0                              what is drivers abstract
1                            why do drivers brake check
2                            why do drivers cancel uber
3     why do drivers drive their car slowly during r...
4                    why do drivers exceed speed limits
5                 why do drivers flash their headlights
6                           why do drivers get so angry
7     why do drivers have responsibility for protect...
8                    why do drivers in nyc honk so much
9                                  what are drivers job
10                       what is drivers knowledge test
11                       why do drivers licenses expire
12                              what is drivers meaning
13                   why do drivers need auto insurance
14    why do drivers of large trucks create extra sp...
15    why do incompatible drivers prevent using memo...
16                        why are drivers quitting uber
17                        why do drivers run red

In [131]:
 df["drivers_raw"].iloc[1]

['what car is baby driver',
 'what car does baby driver drive',
 'what car is beth driving in yellowstone',
 'why are self driving cars bad',
 'why are self-driving cars bad for the environment',
 'does car driving burn calories']